In [1]:
import time
import bs4
import platform
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
wait = 1

In [3]:
# INSTANCE BROWSER
def openBrowser(business):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
        profile_path = '/home/nomadbitcoin/whatsapp_profiles/' + business 
    elif platform.system() == 'Windows':
        driver_path = r'C:\\Users\\Yan\\Desktop\\brisbane_bot\\webdriver\\win\\chromedriver.exe'
        profile_path = "C:\\Users\\Yan\\Desktop\\brisbane_bot\\profiles\\" + business
        print(profile_path)
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
#         driver.implicitly_wait(5) #tempo implicito de espera 1 segundo antes de cada acao
        main_url = 'https://web.whatsapp.com/'
        driver.get(main_url)
        return True
        if wait_connection(driver):
            return True, 'opened in: {}'.format(main_url)
    except Exception as error:
        print(type(error), error)
        return False
    
# PARA LIBERAR SOMENTE APOS TER ABERTO O WHATSAPP
def wait_connection(driver):
    waiting = True
    while waiting:
        try:
            if driver.find_element_by_class_name('Qk8nZ').text == 'Keep your phone connected':
                waiting = False
                return True
                break
#             elif driver.find_element_by_class_name('_13HPh').text == 'Computer not connected':
#                 waiting = False
#                 return False, 'Computer not connected'
        except KeyboardInterrupt:
            waiting = False
            return False
        except:
            time.sleep(10)
            pass

C:\Users\Yan\Desktop\brisbane_bot\profiles\nomadbitcoin


True

In [5]:
# PEGA A UNICA MENSAGEM QUE HA NO CHAT 'TO_SEND' - FUNCIONANDO PARA IMAGENS
def click_forward(content, have_text=False):
    '''
        Deve Haver uma conversa com as mensagens a serem encaminhadas e NADA MAIS,
        pode ser um grupo sozinho ou uma conversa com alguem mas 
        é essencial que não tenha mais nenhuma mensagem na conversa além das mensagens a serem encaminnhadas
    
        o conteudo pode ser imagem, audio, video, ou texto e caso seja imagem ou video pode conter texto
    '''
    
    # se o conteudo a ser encaminhado for imagem
    if content == 'image':
        try:
            div_chat = driver.find_element_by_class_name('_1ays2')
            mensagens = div_chat.find_elements_by_class_name('FTBzM')
            for msg in mensagens:
                try:
                    #verifica se ha imagem a ser verificada e verifica se ja foi verificada
                    src = msg.find_element_by_tag_name('img').get_attribute('src')
                    msg.find_element_by_xpath('//span[@data-icon="forward-chat"]').click()
                    #retorna True para o clique e tambem o conteudo de texto da mensagem, senao somente True
                    text = msg.text[:-8]if have_text == True else None
                    return True, text #os ultimos 8 caracteres sao o horario da mensagem e nao devem ser enviados
                except Exception as error:
                    if 'Unable to locate element' not in str(error):
                        print(type(error), error)
                    pass

        except Exception as error:
            print(type(error), error)

In [6]:
def click_to_send():
    try:
        driver.find_element_by_xpath('//div[@data-animate-btn="true"]').click()
        return True
    except Exception as error:
        print(type(error), error)
        return False

In [7]:
def getChats():
    '''
        Pega a lista de chats apos clicar no botao de encaminhar
    '''
    chats = driver.find_elements_by_class_name('_2UaNq')
    return chats

In [8]:
def isGroup(chat):
    '''
        Rececebe um chat e verifica se eh um grupo
    
        Quado eh grupo nao tera uma div especifica, entao retornara False
    '''
    try:
        div_contact = chat.find_element_by_class_name('_3NWy8')
        return False
    except Exception as error:
        if 'Unable to locate element' not in str(error):
            print(type(error), error)
        else:
            return True

In [9]:
def clickChat(chat):
    try:
        chat.click()
        return True
    except Exception as error:
        return False

def select_contacts(groups=False):
    '''
        Verifica se os chats sao grupo, ira selecionar 5 chats que nao sejam grupos
    '''
    try:
        # caso nao seja para encaminhar para os grupos
        if groups == False:
            selected = 0
            for chat in getChats():
                time.sleep(wait)
                if not isGroup(chat) and clickChat(chat) and selected <=5:
                    print('{} selected: {}'.format(selected, chat.text))
                    selected +=1
                elif selected >=5:
                    break
            return True
        # se for para selecionar grupos eh a mesma coisa mais nao ira verificar antes se eh grupo
        else:
            selected = 0
            for chat in getChats():
                time.sleep(wait)
                if clickChat(chat) and selected <=5:
                    print('{} selected: {}'.format(selected, chat.text))
                    selected +=1
                elif selected >=5:
                    break
            return True
            
    except Exception as error:
        print(type(error), error)

In [10]:
def scrollChats():
    '''
        Ira controlar os scrolls a serem dados na lista de chats
    '''
    pass

In [11]:
# FUNCOES DE NAVEGACAO PELO WHATSAPP COM CLIQUES E TECLAS DO TECLADO
def down_chat(force=False, page_down=True):
    #force = True ira clicar na caixa de pesquisa e reiniciar o processo de clicar nos botoes
    if 'press_down' not in globals() or force == True:
        # CLICA NA CAIXA DE PESQUISA DE CONVERSAS
        driver.find_element_by_class_name('_2zCfw').click()
        
        # PRESSIONA A TECLA DOWN PARA NAVEGAR ENTRE AS CONVERSAS
        global press_down
        press_down = ActionChains(driver)
        press_down.send_keys(Keys.DOWN)
        press_down.perform()
    elif page_down == True:
        press_down.send_keys(Keys.PAGE_DOWN)
        press_down.perform()
    else:
        press_down.perform()

In [45]:
# CLICA EM ENCAMINHAR MENSAGEM
def click_forward():
    '''
        Verifica se a ultima mensagem foi a propria conta que enviou e se eh uma imagem e se o tempo esta valido,
        apos clicar em encaminhar ira aguardar ate que o reloginho de enviando conclua
        tempo valido sera se o horario atual comparado ao tempo que esperou enviar coincide com o horario da imagem enviada
        (para casos em que a conta ja tenha enviado uma imagem antes)
    '''
    pass

In [29]:
# ABRE UMA CONVERSA ESPECIFICA QUE CONTEM O CONTEUDO A SER ENCAMINHADO
def openChat(chat_name):
    '''
        Abre a conversa principal onde esta o conteudo a ser encaminhado
    '''

    try:
        '''Na caixa de pesquisa digita o nome do contato'''
        search_box = driver.find_element_by_class_name('_2zCfw')
        search_box.click()
        search_box.send_keys(Keys.CONTROL + 'a')
        search_box.send_keys(Keys.BACKSPACE)
        search_box.send_keys(chat_name)
        time.sleep(wait)

        ''' Clica e Abre a Conversa'''
        chats = driver.find_elements_by_class_name('KgevS')
        for chat in chats:
            if chat.find_element_by_class_name('_19RFN').get_attribute('title') == chat_name:
                chat.click()
                return True
    except Exception as error:
        print(type(error), error)
        return False

In [41]:
# ENCONTRA O CONTEUDO ESPECIFICO E CLICA EM ENCAMINHAR
def sendContent(content_type):
    '''
        Recebe como parametro um tipo de conteudo e chama a funcao responsavel por clicar em encaminhar aquele conteudo
        
        precisa implementar:
            *documento
            *localizacao
            *contato
    '''
    if content_type == 'audio':
        sendAudioFile()
        return True
    elif content_type == 'image':
        sendImage()
        return True
    elif content_type == 'video':
        sendVideo()
        return True
    elif content_type == 'text':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- TEXT]')
        return False
    elif content_type == 'document':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- DOCUMENT]')
        return False
    elif content_type == 'location':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- LOCATION]')
        return False
    elif content_type == 'contact':
        print('___ >>> sendContent <<< ___ [NEED IMPLEMENT -- CONTACT]')
        return False
    else:
        print('\nError ___ >>> sendContent <<< ___ [function: sendContent()]\n')
        return False

def sendAudioFile():
    '''
        Encontra o arquivo de audio e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('FTBzM')
    
    for message in messages:
        try:
            # se nao der erro eh pq a mensagem eh o audio
            message.find_element_by_class_name('_1N8Pv')
            message.find_element_by_class_name('gxf3C').click()
            return True
        except Exception as error:
    #         print(type(error), error)
            pass
    
    
    # se algo nao der certo ira retornar False
    return False

def sendVideo():
    '''
        Encontra o video e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('FTBzM')
    for message in messages:
        try:
            message.find_element_by_class_name('_3Z-uK')
            message.find_element_by_class_name('gxf3C').click()
            return True
        except Exception as error:
    #         print(type(error), error)
            pass
    
def sendImage():
    '''
        Encontra o video e passa para a funcao encaminhar
    '''
    messages = driver.find_elements_by_class_name('FTBzM')
    for message in messages:
        try:
            message.find_element_by_tag_name('img')
            message.find_element_by_class_name('gxf3C').click()
            return True
        except Exception as error:
#             print(type(error), error)
            pass
    return False

In [43]:
openChat('DIVO')

True

In [44]:
sendContent('video')

True

In [82]:
select_contacts()

passou
4 selected: +55 48 9138-8801

passou
5 selected: +55 51 9134-6735

passou
6 selected: +55 47 9958-0999

passou
7 selected: Mae

passou
8 selected: Gab



(True,
 ['+55 48 9138-8801', '+55 51 9134-6735', '+55 47 9958-0999', 'Mae', 'Gab'])

In [125]:
# SELECIONA UM OU MAIS CONTATOS QUE AINDA NAO ENCAMINHOU PARA ENVIAR A MENSAGEM
def select_contacts(groups=False, just_one=False):
    '''
        Recebe como parametro se deve selecionar um ou mais contatos
        Verifica se os chats sao grupo, ira selecionar 5 chats que nao sejam grupos
    '''
    selected = []
    try:
        # caso nao seja para encaminhar para os grupos
        if groups == False:
            for num, chat in enumerate(getChats()):
                time.sleep(wait/2)
                chat_title = chat.text.split('\n')[0]
                if len(selected) <=4 and not isGroup(chat) and not verifyForwarded(chat_title) and clickChat(chat):
                    print('{} selected: {}\n'.format(num, chat_title))
                    selected.append(chat_title)
                    
                    ''' Se for para selecionar apenas um ira encerrar a funcao apos selecionar um'''
                    if just_one == True:
                        return True
                elif len(selected) >=4:
                    break
            return selected
        # se for para selecionar grupos eh a mesma coisa mas nao ira verificar antes se eh grupo
        else:
            for num, chat in enumerate(getChats()):
                time.sleep(wait/2)
                chat_title = chat.text.split('\n')[0]
                if len(selected) <=4 and not verifyForwarded(chat_title) and clickChat(chat):
                    chat_title = chat.text.split('\n')[0]
                    print('{} selected: {}\n'.format(num, chat_title))
                    selected.append(chat_title)
                    ''' Se for para selecionar apenas um ira encerrar a funcao apos selecionar um'''
                    if just_one == True:
                        return True
                elif len(selected) >=4:
                    break
            return selected
            
    except Exception as error:
        print(type(error), error)

def isGroup(chat):
    '''
        Rececebe um chat e verifica se eh um grupo
    
        Quado eh grupo nao tera uma div especifica, entao retornara False
    '''
    try:
        div_contact = chat.find_element_by_class_name('_3NWy8')
        return False
    except Exception as error:
        if 'Unable to locate element' not in str(error):
            print(type(error), error)
        else:
            return True
        
def getChats():
    '''
        Pega a lista de chats apos clicar no botao de encaminhar
    '''
    chats = driver.find_elements_by_class_name('_2UaNq')
    return chats

def click_to_send(names_to_save):
#     try:
    driver.find_element_by_xpath('//div[@data-animate-btn="true"]').click()
    ''' Salva os nomes dos contatos para os quais ja encaminhou'''
    saveForwarded(names_to_save, 'list')
    return True
#     except Exception as error:
#         print(type(error), error)
#         return False

In [117]:
# SALVA CONTATOS QUE JA INTERAGIU E VERIFICA CONTATOS QUE JA INTERAGIU
def saveForwarded(names_to_save, type_of_send):
    '''
        Salva os contatos para o qual ja encaminhou.
        ha dois tipos de encaminhamento, para uma pessoa ou para mais de uma
        se for mais de uma recebera uma lista de nomes para salvar
    '''
    if type_of_send == 'one':
        with open('forwarded.txt', 'a') as filee:
            filee.write(str(name) + '\n')
        return True
    
    if type_of_send == 'list':
        for name in names_to_save:
            print('name')
            with open('list_forwarded.txt', 'a') as filee:
                    filee.write(str(name) + '\n')
        return True
    
def verifyForwarded(chat_name=None, get_list=False):
    '''
        Verifica se ja encaminhou para aquele contato antes
    '''
    with open('list_forwarded.txt', 'r') as filee:
        talked = filee.read().split('\n')
    
    if chat_name != None:
        return True if chat_name in talked else False
    elif get_list != False:
        return talked

In [128]:
selected_chats = select_contacts(True)
click_to_send(selected_chats)

10 selected: Save Save Save

11 selected: +55 51 9134-6735

12 selected: +55 34 9145-9980

13 selected: +55 11 94659-6959



In [115]:
openBrowser('nomadbitcoin')

C:\Users\Yan\Desktop\brisbane_bot\profiles\nomadbitcoin


True

In [ ]:
#abrir uma conversa especifica
#clicar em encaminhar no conteudo especifico
#selecionar contatos que nao enviou, exceto grupos
#clicar em enviar

#dar scroll selecionando outros na lista
#repetir o processo de onde parou